<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# we upload, in case that we cant parse logs(a lot of time and ram consumption)

# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#copy log file
!cp '/content/drive/MyDrive/zHDFS2/logs_train' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_ntest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_atest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/log2index' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/reduced_embeddings' '/content/'


In [ ]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with open('/content/reduced_embeddings', 'r') as json_file:
    embeddings = json.load(json_file)

In [ ]:
len(embeddings[0])


10

In [ ]:

name = 'logs_train'
window_size = 10
num_sessions = 0
inputs = []
#outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [ embeddings[int(i)] for i in row.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                #outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
#dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float))

Number of sessions(logs_train): 446578
Number of seqs(logs_train): 4704000


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers): #embedding dimensions
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=True,batch_first=True)


    def forward(self, src):

        #src = [src len, batch size]


        outputs, (hidden, cell) = self.rnn(src)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs[:,-1:,:]  #outputs[:,-1:,:]=output of the end of blocks which is concatenated of two directions(we use -1: for keeping the dimensions)

class Decoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers):       #input _dim depends on encoder's output
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=False,batch_first=True)

    def forward(self, src):

        #src = [src len, batch size]
        src = torch.tile(src, (1, 10, 1)) # give the output of encoder to each cell of decoder

        outputs, (hidden, cell) = self.rnn(src)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs

class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):
        encoded = self.encoder(src)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
encoder = Encoder(10,32,1)
decoder = Decoder(64,10,1)
model = Autoencoder(encoder, decoder).to(device)
model

Autoencoder(
  (encoder): Encoder(
    (rnn): LSTM(10, 32, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (rnn): LSTM(64, 10, batch_first=True)
  )
)

In [ ]:
input_data = torch.rand(1, 10, 10).to(device) #(batch,sequence_length,embedding)
output= model(input_data)
print(output.shape)

torch.Size([1, 10, 10])


In [ ]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 14304


In [ ]:
dataloader = DataLoader(dataset, batch_size=256)

# Loss and optimizer
criterion = nn.MSELoss()

In [ ]:
len(dataloader) # regard to batch size

18375

In [ ]:
for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

torch.Size([256, 10, 10])


In [ ]:
def adjust_learning_rate(optimizer, epoch, lr_step=(30, 50), lr_decay_ratio=0.1):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 16
    elif epoch in [1, 2, 3, 4]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [ ]:

num_epochs = 60
input_size = 10 # embedding size

# Train the model
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Epoch [1/100], train_loss: 0.0165
Epoch [2/100], train_loss: 0.0116
Epoch [3/100], train_loss: 0.0095
Epoch [4/100], train_loss: 0.0085
Epoch [5/100], train_loss: 0.0067
Epoch [6/100], train_loss: 0.0055
Epoch [7/100], train_loss: 0.0049
Epoch [8/100], train_loss: 0.0046
Epoch [9/100], train_loss: 0.0043
Epoch [10/100], train_loss: 0.0040
Epoch [11/100], train_loss: 0.0038
Epoch [12/100], train_loss: 0.0036
Epoch [13/100], train_loss: 0.0035
Epoch [14/100], train_loss: 0.0034
Epoch [15/100], train_loss: 0.0033
Epoch [16/100], train_loss: 0.0032
Epoch [17/100], train_loss: 0.0032
Epoch [18/100], train_loss: 0.0031
Epoch [19/100], train_loss: 0.0031
Epoch [20/100], train_loss: 0.0030
Epoch [21/100], train_loss: 0.0030
Epoch [22/100], train_loss: 0.0031
Epoch [23/100], train_loss: 0.0030
Epoch [24/100], train_loss: 0.0028
Epoch [25/100], train_loss: 0.0029
Epoch [26/100], train_loss: 0.0028
Epoch [27/100], train_loss: 0.0028
Epoch [28/100], train_loss: 0.0028
Epoch [29/100], train_loss: 0

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/zHDFS2/Autoencoder_parameters.pth')

In [ ]:
def generate(name):
    window_size = 10
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            line = line + [52] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by 30 a template with zeros vector
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [ ]:
test_normal_loader, test_normal_length = generate('logs_ntest')
test_abnormal_loader, test_abnormal_length = generate('logs_atest')

Number of sessions(logs_ntest): 1091
Number of sessions(logs_atest): 4126


In [ ]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in test_normal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += test_normal_loader[line] # numbers of that set we have
                break
  with torch.no_grad():
      for line in test_abnormal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += test_abnormal_loader[line]
                break
  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [ ]:
threshold = [0.005,0.01,0.015,0.02,0.025,0.03]
for i in threshold:
  evaluation(i)

elapsed_time: 16.644s
false positive (FP): 6829, false negative (FN): 195, Precision: 70.906%, Recall: 98.842%, F1-measure: 82.575%
Finished Predicting
elapsed_time: 21.221s
false positive (FP): 1472, false negative (FN): 1431, Precision: 91.279%, Recall: 91.501%, F1-measure: 91.390%
Finished Predicting
elapsed_time: 26.718s
false positive (FP): 634, false negative (FN): 6368, Precision: 94.290%, Recall: 62.181%, F1-measure: 74.941%
Finished Predicting
elapsed_time: 36.182s
false positive (FP): 379, false negative (FN): 10345, Precision: 94.485%, Recall: 38.562%, F1-measure: 54.770%
Finished Predicting
elapsed_time: 45.598s
false positive (FP): 138, false negative (FN): 11050, Precision: 97.671%, Recall: 34.375%, F1-measure: 50.852%
Finished Predicting
elapsed_time: 52.240s
false positive (FP): 63, false negative (FN): 12625, Precision: 98.527%, Recall: 25.021%, F1-measure: 39.907%
Finished Predicting


In [ ]:
threshold = [0.006,0.007,0.008,0.009,0.01,0.011,0.12,0.13]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.006
elapsed_time: 17.042s
false positive (FP): 6025, false negative (FN): 215, Precision: 73.397%, Recall: 98.723%, F1-measure: 84.197%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.007
elapsed_time: 18.647s
false positive (FP): 5008, false negative (FN): 1006, Precision: 75.969%, Recall: 94.025%, F1-measure: 84.038%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.008
elapsed_time: 19.471s
false positive (FP): 4154, false negative (FN): 1087, Precision: 79.131%, Recall: 93.544%, F1-measure: 85.736%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.009
elapsed_time: 23.250s
false positive (FP): 1805, false negative (FN): 1270, Precision: 89.610%, Recall: 92.458%, F1-measure: 91.012%
Finished Predicting
-----------------

ZeroDivisionError: ignored